In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from spring_example import get_spring_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
from scipy.integrate import solve_ivp
import pickle

# Generate data

In [2]:
training_data = get_spring_data(100)
validation_data = get_spring_data(10)

In [3]:
len(training_data['t'])

289

# Set up model and training parameters

In [6]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 2
params['poly_order'] = 2
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.01
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] ='glorot_normal'
# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5


params['activation'] = 'elu'
params['widths'] = [128,64]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 289
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [7]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'experiment_results_4_'+str(i) + '.pkl'

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    
    # Assuming results_dict and params are dictionaries
    data_to_append = {**results_dict, **params}

    # Create a Series from the data_to_append dictionary
    series_to_append = pd.Series(data_to_append)

    # Save the Series as a pickle file
    with open('experiment_results_4_' + str(i) + '.pkl', 'wb') as file:
        pickle.dump(series_to_append, file)
    

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.02360079623758793, (0.023379704, 2.5406148, 0.18287927, 0.26231954)
   validation loss 0.019102932885289192, (0.018887868, 2.8121438, 0.14367034, 0.26231954)
decoder loss ratio: 0.385364, decoder SINDy loss  ratio: 1.451567
Epoch 100
   training loss 5.528091423911974e-05, (1.6675787e-05, 0.30225518, 0.04270143, 0.2141654)
   validation loss 3.7611443985952064e-05, (1.2308531e-05, 0.21647765, 0.02467476, 0.2141654)
decoder loss ratio: 0.000251, decoder SINDy loss  ratio: 0.249300
Epoch 200
   training loss 1.4457706129178405e-05, (6.9629377e-06, 0.039362457, 0.0053551425, 0.2849074)
   validation loss 1.0434996511321515e-05, (4.7243975e-06, 0.024903312, 0.003232718, 0.2849074)
decoder loss ratio: 0.000096, decoder SINDy loss  ratio: 0.032662
Epoch 300
   training loss 9.548550224280916e-06, (4.7818435e-06, 0.015267706, 0.0020429825, 0.298183)
   validation loss 7.893581823736895e-06, (3.3053207e-06, 0.014604979, 0.0017523649, 0.298183)
d

Epoch 3000
   training loss 3.670980959213921e-06, (8.47736e-07, 0.0010680528, 0.00014223612, 0.26987243)
   validation loss 3.4876497920777183e-06, (6.383676e-07, 0.0014788242, 0.00015323328, 0.26987243)
decoder loss ratio: 0.000013, decoder SINDy loss  ratio: 0.001548
THRESHOLDING: 4 active coefficients
Epoch 3100
   training loss 3.777640358748613e-06, (9.537038e-07, 0.0010683099, 0.00014335736, 0.26988426)
   validation loss 3.6189221646054648e-06, (7.7103215e-07, 0.0014514476, 0.00015295007, 0.26988426)
decoder loss ratio: 0.000016, decoder SINDy loss  ratio: 0.001545
Epoch 3200
   training loss 3.736498001671862e-06, (9.181296e-07, 0.0010053683, 0.00013857962, 0.269881)
   validation loss 3.5334664971742313e-06, (6.906239e-07, 0.0014044736, 0.00014761758, 0.269881)
decoder loss ratio: 0.000014, decoder SINDy loss  ratio: 0.001491
Epoch 3300
   training loss 4.372773673821939e-06, (1.5553622e-06, 0.0010302148, 0.00013446846, 0.26986665)
   validation loss 3.840520548692439e-06, (1

Epoch 900
   training loss 4.5263621473168314e-07, (3.8361017e-07, 0.0006383364, 7.4218435e-05, 0.27586928)
   validation loss 4.20682880530876e-07, (3.414378e-07, 0.0007890817, 7.958198e-05, 0.27586928)
decoder loss ratio: 0.000007, decoder SINDy loss  ratio: 0.000804
Epoch 1000
   training loss 1.010953042168694e-06, (9.394583e-07, 0.00067002955, 7.5986565e-05, 0.27589867)
   validation loss 9.469589485888719e-07, (8.6864094e-07, 0.00077753456, 7.88825e-05, 0.27589867)
decoder loss ratio: 0.000018, decoder SINDy loss  ratio: 0.000797
